In [ ]:
import sys
if '../' != sys.path[0]: 
    sys.path.insert(0, '../')
import numpy as np
import pandas as pd
import datapane as dp
import json
import os
import re


from auto_assist.lib import is_pinyin, contain_chinese

from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

In [ ]:
def load_data(data_dir: str):
    gs_profile_file = os.path.join(data_dir, 'gs_profiles.jsonl')
    data = []
    with open(gs_profile_file, 'r', encoding='utf8') as fp:
        for line in fp:
            data.append(json.loads(line))
    return data

def get_cited(text: str):
    line = text.splitlines()[1]
    _, c_all, c_5yr = line.split()
    return int(c_all), int(c_5yr)

def is_chinese(name):
    if contain_chinese(name):
        return True
    if is_pinyin(name.rsplit(None, 1)[-1]):
        return True
    return False

def screening(text, whitelist, blacklist):
    for token in whitelist:
        if token in text:
            return True, token
    for token in blacklist:
        if token in text:
            return False, token
    return True, None

def get_email_from_brief(brief):
    match = re.search(r'Verified email at (\S+)', brief)
    return match.group(1) if match else None

In [ ]:
data = load_data('../out')

In [ ]:
whitelist = ['computational', 'quantum chemistry', 'dft', 'density function', 'robot']
blacklist = ['biophysics']


candidates = []
for obj in data:
    full_text = '\n'.join(obj['articles'] + obj['tags']).lower() 
    if not is_chinese(obj['name']):
        continue    
    accept, rule = screening(full_text, whitelist, blacklist)
    if not accept:
        continue
    candidates.push(obj)

# write xml
for i, obj in enumerate(candidates):    
    email = get_email_from_brief(obj['brief'])
    c_all, c_5yr = get_cited(obj['cited_stats'])
    c_5yr_ratio = c_5yr / c_all if c_all > 0 else 0
    
    

    
    

